#### Import

In [60]:
import numpy as np
import pandas as pd
from scipy.stats import mielke, norm, gamma
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import gaussian_kde
from rpy2 import robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import vectors
stats = importr('stats')

#### Functions

In [61]:
def R_density(data):
    def f(arg):
        column = vectors.FloatVector(list(data))
        output = stats.density(column, n=2000)
        xx = np.array(output[0])
        yy = np.array(output[1])
        
        index = np.argmin(np.abs(xx - arg))
        return yy[index]
    return f


def silverman_bandwidth(data):
    if len(data) < 2:
        raise ValueError("Need at least 2 data points")
    
    std = np.std(data)
    IQR = np.quantile(data, .75) - np.quantile(data, .25)
    A = min(std, IQR / 1.34)
    return 0.9 * A * len(data)**(-0.2) / std
    
        
def H(tau, y, q):
    yL = y[:tau]
    yR = y[tau:]
    
    muL = float(np.quantile(yL, q))
    muR = float(np.quantile(yR, q))
    
    fL = R_density(yL)(muL)
    fR = R_density(yR)(muR)
    
    kappaLsq = q * (1 - q) / fL**2
    kappaRsq = q * (1 - q) / fR**2
    
    return (muL - muR) / (kappaLsq / tau + kappaRsq / (n - tau)) ** (1 / 2)

#### Data

In [62]:
n = 95
y = np.random.randn(n)

#### Recreate Figure 2

In [63]:
taus = np.arange(n)
Hs50 = np.zeros(n)
Hs75 = np.zeros(n)

for i in range(4, n - 4):
    tau = taus[i]
    Hs50[i] = H(tau, y, .5)
    Hs75[i] = H(tau, y, .75)

In [64]:
plt.plot(x[5:-4], Hs50[5:-4])
plt.plot(x[5:-4], Hs75[5:-4])
plt.show()

#### Sandbox

In [32]:
n = 10
data = np.random.beta(a=3.5, b=.8, size=n)

In [33]:
def silverman_bandwidth(data):
    if len(data) < 2:
        raise ValueError("Need at least 2 data points")
    
    std = np.std(data)
    IQR = np.quantile(data, .75) - np.quantile(data, .25)
    A = min(std, IQR / 1.34)
    return 0.9 * A * len(data)**(-0.2) / std

In [34]:
stats = importr('stats')
column = vectors.FloatVector(list(data))
output = stats.density(column)
xx = np.array(output[0])
yy = np.array(output[1])

In [35]:
bw = silverman_bandwidth(data)
kde = gaussian_kde(data, bw_method=bw)

In [36]:
plt.plot(xx, yy)
plt.plot(xx, kde(xx))
plt.show()

In [ ]:
plt.plot(xRR, yRR)
plt.plot(np.linspace(0, 1.5, 200), gaussian_kde(data)(np.linspace(0, 1.5, 200)))
plt.show()

In [ ]:
gaussian_kde(data)(.5)